In [ ]:
#| default_exp evaluation

# Datasets Evaluation

In [ ]:
#| export
from typing import Any, Callable, List, Optional

import numpy as np
import pandas as pd
from fugue import transform

In [ ]:
#| export
def _evaluate(
        df: pd.DataFrame, 
        metrics: List[Callable],
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        level: Optional[List] = None,
    ) -> pd.DataFrame:
    eval_ = {}
    cols_to_rm = '|'.join([id_col, time_col, target_col, 'cutoff', 'lo', 'hi'])
    has_cutoff = 'cutoff' in df.columns
    models = df.loc[:, ~df.columns.str.contains(cols_to_rm)].columns
    for model in models:
        eval_[model] = {}
        for metric in metrics:
            eval_[model][metric.__name__] = metric(df[target_col], df[model])
    eval_df = pd.DataFrame(eval_).rename_axis('metric').reset_index()
    if has_cutoff:
        eval_df.insert(0, 'cutoff', df['cutoff'].iloc[0])
    eval_df.insert(0, id_col, df[id_col].iloc[0])
    return eval_df

In [ ]:
#| export
def _schema_evaluation(
        df: pd.DataFrame, 
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
    ):
    cols_to_rm = '|'.join([id_col, time_col, target_col, 'cutoff', 'lo', 'hi'])
    has_cutoff = 'cutoff' in df.columns
    models = df.loc[:, ~df.columns.str.contains(cols_to_rm)].columns
    str_models = ','.join([f"{model}:double" for model in models])
    dtypes = df.dtypes
    id_col_type = dtypes.loc[id_col]
    if id_col_type == 'category':
        raise NotImplementedError(
            'Use of `category` type to identify each time series is not yet implemented. '
            f'Please transform your {id_col} to string to continue.'
        )
    id_col_type = 'string' if id_col_type == 'object' else id_col_type
    cutoff_col_type = ''
    if has_cutoff:
        cutoff_col_type = f"{dtypes.loc['cutoff']}".replace('64[ns]', '')
    schema = (
        f'{id_col}:{id_col_type},metric:string,'
        + (f'cutoff:{cutoff_col_type},' if has_cutoff else '')
        + str_models
    )
    return schema

In [ ]:
#| hide
from fastcore.test import test_eq
from datasetsforecast.m4 import M4Evaluation

In [ ]:
#| hide
fforma_url = 'https://github.com/Nixtla/m4-forecasts/raw/master/forecasts/submission-245.zip'
M4Evaluation.load_benchmark('data', 'Hourly', fforma_url)
eval_test = pd.read_csv('data/m4/datasets/submission-245.csv').query("id.str.startswith('H')")
eval_test = eval_test.set_index('id')
eval_test.columns = list(range(1, 49))
eval_test = eval_test.unstack().reset_index()
eval_test.columns = ['ds', 'unique_id', 'FFORMA']
eval_test = eval_test[['unique_id', 'ds', 'FFORMA']].sort_values(['unique_id', 'ds'])
eval_test['y'] = 0

In [ ]:
#| hide
test_eq(
    _schema_evaluation(eval_test),
    'unique_id:string,metric:string,FFORMA:double'
)

In [ ]:
#| export
def evaluate_forecasts(
        df: pd.DataFrame,
        metrics: List[Callable],
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        level: Optional[List] = None,
        agg_fn: Optional[Callable] = None,
        agg_by: Optional[str] = None,
        engine: Any = None,
        kwargs_engine: Any = dict(),
    ) -> pd.DataFrame:
    evaluation_df = transform(
        df, 
        _evaluate, 
        engine=engine, 
        params=dict(
            metrics=metrics,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            level=level,
        ), 
        schema=_schema_evaluation(
            df, 
            id_col=id_col, 
            time_col=time_col, 
            target_col=target_col,
        ), 
        partition=dict(by='unique_id')
    )
    return evaluation_df

In [ ]:
#| hide
from datasetsforecast.losses import smape

In [ ]:
evaluate_forecasts(eval_test, metrics=[smape]).groupby('metric').agg(np.mean)

/var/folders/7x/1l3vkh3x4_q3s4r36b60s8f00000gn/T/ipykernel_32058/2552758337.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  evaluate_forecasts(eval_test, metrics=[smape]).groupby('metric').agg(np.mean)


,FFORMA
metric,
smape,199.969807
